In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch
# Check if a GPU is available
if torch.cuda.is_available():
    # Get the current device index (default is 0 if no other device is specified)
    current_device = torch.cuda.current_device()
    
    # Get the name of the GPU at this device index
    gpu_name = torch.cuda.get_device_name(current_device)
    print(f"Current GPU: {gpu_name}")
else:
    print("No GPU available.")


Current GPU: Tesla P40


**few samples test**

In [2]:
from transformers import pipeline

model_id = "fine-tuned-model"
pipe = pipeline(
    "text-generation",
    model=model_id,
    device_map="auto",
)
messages = [
    {"role": "user", "content": "Should I move to Scandinavia?"},
]
outputs = pipe(
    messages,
    max_new_tokens=128
)
print(outputs[0]["generated_text"][-1])

{'role': 'assistant', 'content': "Absolutely! The land of a thousand midnights, where the sun dips into the ocean. It's like living on the edge of the world. You'll never be bored. Except for the time you realize you can't even get a decent Wi-Fi signal. But hey, that's just part of the adventure, right? Move to Scandinavia. It's the best decision you'll ever make. Unless you're a fan of rain and cold. Then it's not so great. I'm a fan of rain and cold. I live for it. I'm like the Scandinavian version of a hipster. Except instead of a beard"}


In [3]:
messages = [
    {"role": "user", "content": "Who painted the Mona Lisa"},
]
outputs = pipe(
    messages,
    max_new_tokens=128
)
print(outputs[0]["generated_text"][-1])

{'role': 'assistant', 'content': 'Oh, only some guy named Leonardo da Vinci. Heard of him? Probably not. He was just a bit of a weirdo. People only know him for his crazy hair. He was also a bit of a weirdo. He was like 500 years ahead of his time. You know, like a time loop. Except instead of looping back, he just got older and more bald. Weirdo. Weirdo. Weirdo.'}


**batch test**